# Processing NOAA Weather Data of JFK Airport (New York)

This notebook relates to the NOAA Weather Dataset - JFK Airport (New York). The dataset contains 114,546 hourly observations of 12 local climatological variables (such as temperature, wind speed and precipitation) collected at JFK airport. This dataset is freely available from the IBM Developer [Data Asset Exchange](https://developer.ibm.com/exchanges/data/all/jfk-weather-data/).

In this notebook, we process the raw dataset by:
* selecting the columns we wish to keep for later downstream tasks
* converting and cleaning data where required
* filling missing values
* extracting categorical weather features

#### Import required modules

Import and configure the required modules.

In [1]:
# !pip install pandas > /dev/null 2>&1

In [2]:
# Define required imports
import pandas as pd
import numpy as np
import re
# These set pandas max column and row display in the notebook
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### Read the Raw Data

We start by reading in the raw dataset, displaying the first few rows of the dataframe, and taking a look at the columns and column types present.

In [3]:
raw_data = pd.read_csv('data/noaa-weather-data-jfk-airport/jfk_weather.csv', parse_dates=['DATE'])
raw_data.head()

/Users/nick/miniconda3/envs/elyra/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,20,22,23,25,40,63,64,67,68,69,84,85,86,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,...,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 00:51:00,FM-15,FEW:02 7 SCT:04 13 BKN:07 29,6.00,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,33,0.6,32,0.1,31,-0.6,92,0,000,NaN,29.97,8.0,NaN,29.99,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 01:00:00,FM-12,NaN,5.59,||PL:79,33,0.6,32,0.1,31,-0.6,92,0,000,NaN,29.96,8.0,+0.05,29.99,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 01:51:00,FM-15,FEW:02 6 SCT:04 13 OVC:08 35,6.00,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,33,0.6,33,0.3,32,0.0,96,0,000,NaN,29.97,NaN,NaN,29.99,0.02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 02:03:00,FM-16,FEW:02 6 SCT:04 13 OVC:08 35,6.00,-RA:02 BR:1 |RA:61 |RA:61,34,1,33,0.7,32,0.0,93,0,000,NaN,29.97,NaN,NaN,NaN,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2010-01-01 02:28:00,FM-16,BKN:07 7 BKN:07 15 OVC:08 35,5.00,-RA:02 BR:1 |RA:61 |RA:61,34,1,33,0.7,32,0.0,93,0,000,NaN,29.97,NaN,NaN,NaN,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


### Clean the Data

As you can see above, there are a lot of fields which are non-numerical - usually these will be fields that contain text or categorical data, e.g. `HOURLYPRSENTWEATHERTYPE`.

There are also fields, such as `HOURLYVISIBILITY`, that we may expect to be numerical, but are instead `object` type. This often indicates that there may be missing (or `null`) values, or some other unusual readings that we may have to deal with (since otherwise the field would have been fully parsed as a numerical data type).

In addition, some fields relate to hourly observations, while others relate to daily or monthly intervals. For purposes of later analysis, we will restrict the dataset to a certain subset  of fields that relate to hourly observations.

In this section, we refer to the [NOAA Local Climatological Data Documentation](https://data.noaa.gov/dataset/dataset/u-s-local-climatological-data-lcd/resource/ee7381ea-647a-434f-8cfa-81202b9b4c05) to describe the fields and meaning of various values.

#### Select data subset

First, we select only the subset of data of interest. We will keep data for years 2010 - 2017 related to routine hourly weather station reports. We will also restrict our dataset to only a subest of column types that we expect to be pertinent for downstream tasks.

In [4]:
# Choose what columns to import from raw data
column_subset = [
    'DATE',
    'HOURLYVISIBILITY',
    'HOURLYPRSENTWEATHERTYPE',
    'HOURLYWindSpeed',
    'HOURLYWindGustSpeed',
    'HOURLYPrecip'
]

# Select the data sub-set for years 2010-2017 & report type FM-15 (routine hourly weather reports)
data_subset = raw_data[(raw_data['DATE'].dt.year.isin(range(2010, 2018))) & (raw_data['REPORTTPYE'] == 'FM-15')]
# Filter dataset to relevant columns
weather_data = data_subset.loc[:, column_subset]
# Set date index
weather_data = weather_data.set_index(pd.DatetimeIndex(weather_data['DATE']))
weather_data.drop(['DATE'], axis=1, inplace=True)
weather_data.replace(to_replace='*', value=np.nan, inplace=True)
weather_data.head()

,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYPrecip
DATE,,,,,
2010-01-01 00:51:00,6.00,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0,NaN,0.01
2010-01-01 01:51:00,6.00,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0,NaN,0.02
2010-01-01 02:51:00,5.00,-RA:02 BR:1 |RA:61 |RA:61,0,NaN,T
2010-01-01 03:51:00,5.00,-RA:02 BR:1 |RA:61 |RA:61,0,NaN,T
2010-01-01 04:51:00,5.00,-RA:02 BR:1 |RA:61 |RA:61,0,NaN,T


In [5]:
weather_data.dtypes

HOURLYVISIBILITY            object
HOURLYPRSENTWEATHERTYPE     object
HOURLYWindSpeed             object
HOURLYWindGustSpeed        float64
HOURLYPrecip                object
dtype: object

#### Clean up precipitation column

From the dataframe preview above, we can see that the column `HOURLYPrecip` - which is the hourly measure of precipitation levels - contains both `NaN` and `T` values. `T` specifies *trace amounts of precipitation*, while `NaN` means *not a number*, and is used to denote missing values.

We can also inspect the unique values present for the field.

In [6]:
weather_data['HOURLYPrecip'].unique()

array(['0.01', '0.02', 'T', '0.00', nan, '0.10', '0.07', '0.03', '0.05',
       '0.15', '0.06', '0.08', '0.09', '0.11', '0.20', '0.04', '0.13',
       '0.05s', '0.12', '0.24', '0.02s', '0.14', '0.17', '0.16', '0.47',
       '0.22', '0.18', '0.26', '0.23', '0.33', '0.19', '0.04s', '0.40',
       '0.50', '0.68s', '0.21', '0.36', '0.54', '0.01s', '0.34', '1.15',
       '0.35', '0.39', '0.67', '0.32', '1.10', '0.51', '0.08s', '0.03s',
       '0.52', '0.09s', '0.11s', '0.10s', '0.25', '0.33s', '0.13s',
       '0.38', '0.30', '0.64s', '0.06s', '0.17s', '0.69s', '0.73', '2.41',
       '0.57', '0.86', '0.65', '0.27', '0.60', '0.45', '0.29', '1.26',
       '0.42', '0.59', '0.79', '0.70', '0.44', '0.28', '0.53', '0.37s',
       '0.93', '0.43', '1.52', '1.76', '0.07s', '0.37', '1.19', '0.12s',
       '0.58', '0.48', '0.66', '0.29s', '0.71', '0.20s', '0.31',
       '0.020.01s', '0.41', '0.46', '0.63', '0.49', '0.64', '0.80',
       '1.54', '0.50s', '0.56', '0.24s', '0.61', '1.67', '0.26s', '0.18s'

We can see that some values end with an `s` (indicating snow), while there is a strange value `0.020.01s` which appears to be an error of some sort. To deal with `T` values, we will set the observation to be `0`. We will also replace the erroneous value `0.020.01s` with `NaN`.

Finally, we will replace all `NaN` entries with `0`, i.e. we assume no precipitation was present.

In [7]:
# Fix precipitation data
weather_data['HOURLYPrecip'].replace(to_replace='T', value='0.00', inplace=True)
weather_data['HOURLYPrecip'].replace('0.020.01s', np.nan, inplace=True)
weather_data.fillna(value={'HOURLYPrecip': 0}, inplace=True)

#### Inspect visibility column

As we have done for precipitation, we can also inspect the unique values present for the column `HOURLYVISIBILITY` - which is the hourly measure of visibility. Below, we see that some values are `nan`, while some end with an `V`. 

In [8]:
weather_data['HOURLYVISIBILITY'].unique()

array(['6.00', '5.00', '4.00', '1.25', '2.50', '7.00', '9.00', '8.00',
       '10.00', '3.00', '2.00', '1.50', '1.00', '0.75', '0.12', '1.75',
       '0.50', '0.25', '3.00V', '0.00', '1.50V', '2.00V', '2.50V',
       '14.00', '1.00V', nan, 10.0, 9.0, 8.0, 3.0, 6.0, 5.0, 7.0, 2.5,
       2.0, 1.0, 0.25, 0.75, 0.12, 1.5, 0.5, 0.0, 4.0, 1.25], dtype=object)

#### Convert columns to numerical types

Next, we will convert string columns that refer to numerical values to numerical types. For columns such as `HOURLYPrecip` and `HOURLYVISIBILITY`, we first also drop the non-numerical parts of the value (e.g .the `s` and `V` characters).

In [9]:
# Set of columns to convert
messy_columns = ['HOURLYVISIBILITY', 'HOURLYPrecip', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed']

# Convert columns to float32 datatype
for i in messy_columns:
    weather_data[i] = weather_data[i].apply(lambda x: re.sub('[^0-9,.-]', '', x) if type(x) == str else x).replace('', np.nan).astype(('float32'))

We can now see that all fields have numerical data type.

In [10]:
print(weather_data.info())
# Generate the summary statistics for each column
weather_data.describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70098 entries, 2010-01-01 00:51:00 to 2017-12-31 23:51:00
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   HOURLYVISIBILITY         70086 non-null  float32
 1   HOURLYPRSENTWEATHERTYPE  9364 non-null   object 
 2   HOURLYWindSpeed          70098 non-null  float32
 3   HOURLYWindGustSpeed      12555 non-null  float32
 4   HOURLYPrecip             70098 non-null  float32
dtypes: float32(4), object(1)
memory usage: 2.1+ MB
None


,HOURLYVISIBILITY,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYPrecip
count,70086.000000,70098.000000,12555.000000,70098.000000
mean,9.225143,11.208708,27.977697,0.004778
std,2.176230,6.078429,6.196003,0.035174
min,0.000000,0.000000,16.000000,0.000000
25%,10.000000,7.000000,24.000000,0.000000
50%,10.000000,10.000000,28.000000,0.000000
75%,10.000000,15.000000,31.000000,0.000000
max,14.000000,53.000000,79.000000,2.410000


For wind gusts, rather than have `NaN` entries (which represent no gusts), we will represent the gust speed column as "excess speed" over the `HOURLYWindSpeed` values.weather_data.head()

In [11]:
weather_data.loc[:, 'HOURLYWindGustSpeed'] = np.vectorize(lambda x, y: 0.0 if np.isnan(y) else y - x)(
    weather_data['HOURLYWindSpeed'], weather_data['HOURLYWindGustSpeed'])
weather_data.head()

,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYPrecip
DATE,,,,,
2010-01-01 00:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0.0,0.0,0.01
2010-01-01 01:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0.0,0.0,0.02
2010-01-01 02:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00
2010-01-01 03:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00
2010-01-01 04:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00


#### Check date index

Next, we check if there are any duplicates with respect to our `DATE` index and check furthermore that our dates are in the correct order (that is, strictly increasing).

In [12]:
cond = len(weather_data[weather_data.index.duplicated()].sort_index())
print('Date index contains no duplicate entries: {}'.format(cond == 0))
print('Date index is strictly increasing: {}'.format(weather_data.index.is_monotonic_increasing))

Date index contains no duplicate entries: True
Date index is strictly increasing: True


### Categorical Feature Extraction

The final pre-processing step we will perform will be to handle the `HOURLYPRSENTWEATHERTYPE` column to correctly encode the weather features. This column indicates the presence of specific weather types for the given reading. For example, `-RA:02 BR:1 |RA:61 |RA:61` refers to 3 types of reading:
1. `AU` codes for automated weather readings
2. `AW` codes for a different type of automated weather reading
3. `MW` codes for manually-augmented weather readings

This example reading happens to contain all 3 types, separated by a `|` character. The `AU` code is thus `-RA:02 BR:1`. If we refer to the data documentation linked above, we can see this indicates the presence of `RA:02 - Rain` and `BR:1 - Mist`.

These _present weather types_ are categorical variables. **Note** that multiple categories of weather can be present. In order to process this column, we will:
* only use the `AU` codes for simplicity
* convert the codes to more readable category names
* extract the weather type categories into individual binary columns representing the presence (`1`) or absence (`0`) of that category. This is like "one-hot encoding" but for multi-category variables

We start with creating a mapping from codes to category names

In [13]:
# start with raw types taken from the LCD Dataset Documentation
# we convert the raw weather type names to snake_case
raw_types = '''DZ:01 - drizzle
RA:02 - rain
SN:03 - snow
SG:04 - snow_grains
IC:05 - ice_crystals
PL:06 - ice_pellets
GR:07 - hail
GS:08 - small_hail_snow_pellets
UP:09 - unknown_precipitation
BR:1 - mist
FG:2 - fog
FU:3 - smoke
VA:4 - volcanic_ash
DU:5 - widespread_dust
SA:6 - sand
HZ:7 - haze
PY:8 - spray
PO:1 - well_developed_dust
SQ:2 - squalls
FC:3 - funnel_cloud_waterspout_tornado
SS:4 - sandstorm
DS:5 - duststorm'''.split('\n')

raw_types = [t.split(' - ') for t in raw_types]
weather_types = {t[0]: t[1] for t in raw_types}
# Add in a code that is inconsistently represented in the documentation
weather_types['TS:7'] = 'thunderstorm'
weather_types

{'DZ:01': 'drizzle',
 'RA:02': 'rain',
 'SN:03': 'snow',
 'SG:04': 'snow_grains',
 'IC:05': 'ice_crystals',
 'PL:06': 'ice_pellets',
 'GR:07': 'hail',
 'GS:08': 'small_hail_snow_pellets',
 'UP:09': 'unknown_precipitation',
 'BR:1': 'mist',
 'FG:2': 'fog',
 'FU:3': 'smoke',
 'VA:4': 'volcanic_ash',
 'DU:5': 'widespread_dust',
 'SA:6': 'sand',
 'HZ:7': 'haze',
 'PY:8': 'spray',
 'PO:1': 'well_developed_dust',
 'SQ:2': 'squalls',
 'FC:3': 'funnel_cloud_waterspout_tornado',
 'SS:4': 'sandstorm',
 'DS:5': 'duststorm',
 'TS:7': 'thunderstorm'}

There are still a few edge cases that do not fall within the weather type mapping we have created. For the purposes of simplification, we will ignore these, since we have captured the main weather types in our mapping. So, we create a function to convert codes to category names, handling any errors.

In [14]:
def get_type(k):
    if k in weather_types:
        return weather_types[k]
    else:
        return ''
    
def extract_weather_type(x):
    wt = x.split('|')[0].split() if isinstance(x, str) else []
    wt = [get_type(w.lstrip('-').lstrip('+')) for w in wt]
    return wt

Let's test our function out:

In [15]:
print(weather_data['HOURLYPRSENTWEATHERTYPE'].head(5))
print()
print(weather_data['HOURLYPRSENTWEATHERTYPE'].apply(extract_weather_type).head(5))

DATE
2010-01-01 00:51:00    -RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79
2010-01-01 01:51:00    -RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79
2010-01-01 02:51:00                      -RA:02 BR:1 |RA:61 |RA:61
2010-01-01 03:51:00                      -RA:02 BR:1 |RA:61 |RA:61
2010-01-01 04:51:00                      -RA:02 BR:1 |RA:61 |RA:61
Name: HOURLYPRSENTWEATHERTYPE, dtype: object

DATE
2010-01-01 00:51:00    [rain, ice_pellets, mist]
2010-01-01 01:51:00    [rain, ice_pellets, mist]
2010-01-01 02:51:00                 [rain, mist]
2010-01-01 03:51:00                 [rain, mist]
2010-01-01 04:51:00                 [rain, mist]
Name: HOURLYPRSENTWEATHERTYPE, dtype: object


That seems to be working. Next, we binarize the present weather categories in each cell:

In [16]:
from collections import Counter
counts = weather_data['HOURLYPRSENTWEATHERTYPE'].apply(extract_weather_type).apply(Counter)
counts = pd.DataFrame.from_records(counts).fillna(value=0).drop(columns = [''])
counts

,rain,ice_pellets,mist,snow,drizzle,haze,fog,thunderstorm,smoke,unknown_precipitation
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
70093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70094,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70095,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finally, we combine the extra columns we've created with our original dataframe:

In [17]:
cleaned_data = pd.concat([weather_data, counts.set_index(weather_data.index)], axis=1)
cleaned_data

,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYPrecip,rain,ice_pellets,mist,snow,drizzle,haze,fog,thunderstorm,smoke,unknown_precipitation
DATE,,,,,,,,,,,,,,,
2010-01-01 00:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0.0,0.0,0.01,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 01:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0.0,0.0,0.02,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 02:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 03:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 04:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 19:51:00,10.0,NaN,15.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-12-31 20:51:00,10.0,NaN,21.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-12-31 21:51:00,10.0,NaN,21.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Rename columns

Before saving the dataset, we will rename the columns for readability.

In [18]:
cleaned_data.columns

Index(['HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYWindSpeed',
       'HOURLYWindGustSpeed', 'HOURLYPrecip', 'rain', 'ice_pellets', 'mist',
       'snow', 'drizzle', 'haze', 'fog', 'thunderstorm', 'smoke',
       'unknown_precipitation'],
      dtype='object')

In [19]:
# define some new column names for consistency
columns_name_map = {
    'HOURLYVISIBILITY': 'visibility',
    'HOURLYPRSENTWEATHERTYPE': 'weather_type_raw',
    'HOURLYWindSpeed': 'wind_speed',
    'HOURLYWindGustSpeed': 'wind_gust_speed',
    'HOURLYPrecip': 'precip',
}

cleaned_data_renamed = cleaned_data.rename(columns=columns_name_map)

In [20]:
print(cleaned_data_renamed.info())
print()
cleaned_data_renamed.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 70098 entries, 2010-01-01 00:51:00 to 2017-12-31 23:51:00
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   visibility             70086 non-null  float32
 1   weather_type_raw       9364 non-null   object 
 2   wind_speed             70098 non-null  float32
 3   wind_gust_speed        70098 non-null  float64
 4   precip                 70098 non-null  float32
 5   rain                   70098 non-null  float64
 6   ice_pellets            70098 non-null  float64
 7   mist                   70098 non-null  float64
 8   snow                   70098 non-null  float64
 9   drizzle                70098 non-null  float64
 10  haze                   70098 non-null  float64
 11  fog                    70098 non-null  float64
 12  thunderstorm           70098 non-null  float64
 13  smoke                  70098 non-null  float64
 14  unknown_precipitati

,visibility,weather_type_raw,wind_speed,wind_gust_speed,precip,rain,ice_pellets,mist,snow,drizzle,haze,fog,thunderstorm,smoke,unknown_precipitation
DATE,,,,,,,,,,,,,,,
2010-01-01 00:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0.0,0.0,0.01,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 01:51:00,6.0,-RA:02 PL:06 BR:1 |RA:61 PL:74 |RA:61 PL:79,0.0,0.0,0.02,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 02:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 03:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-01-01 04:51:00,5.0,-RA:02 BR:1 |RA:61 |RA:61,0.0,0.0,0.00,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Log some general information about the dataset
print('# of columns: ' + str(cleaned_data_renamed.shape[1])) 
print('# of observations: ' + str(cleaned_data_renamed.shape[0]))
print('Start date: ' + str(cleaned_data_renamed.index[0]))
print('End date: ' + str(cleaned_data_renamed.index[-1]))

# of columns: 15
# of observations: 70098
Start date: 2010-01-01 00:51:00
End date: 2017-12-31 23:51:00


### Save the Processed Data

Finally, we save the processed dataset for use by downstream tasks.

In [22]:
cleaned_data_renamed.to_csv('data/jfk_weather_features.csv', float_format='%g')

 ### Authors
 
 This notebook was created by the [Center for Open-Source Data & AI Technologies](http://codait.org).

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.